# State

In [6]:
import numpy as np
import cupy as cp
from scipy.linalg import inv, expm
import networkx as nx
import time
import cupyx.scipy.linalg as cpx_scipy
import sympy as sp
import cupy as cp


from __future__ import annotations
from typing import Union

import json

from qwak.Errors import StateOutOfBounds, NonUnitaryState, MissingNodeInput
from utils.PerfectStateTransfer import isStrCospec, checkRoots, swapNodes, getEigenVal

from utils.jsonTools import json_matrix_to_complex, complex_matrix_to_json

class CupyState:
    def __init__(self, n: int, nodeList: list = None,
                 customStateList: list = None) -> None:
        """Object is initialized with a mandatory user-inputted dimension, an optional
        stateList parameter which will be used to create the amplitudes for each node in the state
        and an internal stateVec which will be a cupy ndarray representing the column vector.

        Parameters
        ----------
        n : int
            Desired dimension of the state.
        nodeList : list, optional
            List containing what nodes will have uniform superposition in the state, by default None.
        customStateList : list, optional
            Custom amplitudes for the state, by default None.
        """
        self._n = n
        if nodeList is None:
            self._nodeList = cp.array([])  # Use cupy array
        else:
            self._nodeList = cp.array(nodeList)  # Ensure it's a cupy array

        if customStateList is None:
            self._customStateList = cp.array([])  # Use cupy array
        else:
            self._customStateList = cp.array(customStateList)  # Ensure it's a cupy array

        self._stateVec = cp.zeros((self._n, 1), dtype=complex)  # Create a cupy array with zeros

    def buildState(
            self,
            nodeList: list = None,
            customStateList: list = None) -> None:
        """Builds state vector from state list, by creating a balanced superposition of all
        nodes in the nodeList.
        This will be changed in the future to make nodeList make more sense.

        Parameters
        ----------
        nodeList : list, optional
            List containing what nodes will have uniform superposition in the state, by default None.
        customStateList : list, optional
            Custom amplitudes for the state, by default None.
        """
        if nodeList is not None:
            self.resetState()
            self._nodeList = cp.array(nodeList, dtype=complex)  # Ensure it's a cupy array
        if customStateList is not None:
            self.resetState()
            self._customStateList = cp.array(customStateList, dtype=complex)  # Ensure it's a cupy array
        if self._customStateList:
            self._checkUnitaryStateList(self._customStateList)
            for customState in self._customStateList:
                self._checkStateOutOfBounds(customState[0])
                self._stateVec[customState[0]] = customState[1]
        else:
            nodeAmp = cp.sqrt(len(self._nodeList))
            print(f"NodeList :{self._nodeList}")
            for state in self._nodeList:
                print(f"Type of state:{type(state)}")
                state = int(state)
                self._checkStateOutOfBounds(state)
                print(f"State:{state}")
                print(f"Type of state:{type(state)}")
                print(f"Type of stateVec:{type(self._stateVec)}")
                
                self._stateVec[state] = 1 / nodeAmp

    def _checkStateOutOfBounds(self, node: int) -> None:
        """Checks if the state is out of bounds for the system.

        Parameters
        ----------
        node : int
            Node to check.

        Raises
        ------
        StateOutOfBounds
            Out of bounds exception.
        """
        if node >= self._n:
            raise StateOutOfBounds(
                f"State {node} is out of bounds for system of size {self._n} ([0-{self._n - 1}])."
            )

    def _checkUnitaryStateList(self, customStateList) -> None:
        """Checks if the sum of the square of the amplitudes is 1.

        Parameters
        ----------
        customStateList : list
            Custom state list.

        Raises
        ------
        NonUnitaryState
            Non unitary state exception.
        """
        unitaryState = 0
        for state in customStateList:
            unitaryState += cp.abs(state[1]) ** 2
        unitaryState = round(unitaryState, 5)
        if unitaryState != float(1):
            raise NonUnitaryState(
                f"The sum of the square of the amplitudes is -- {unitaryState} -- instead of 1."
            )

    def herm(self) -> cp.ndarray:
        """Returns the Hermitian conjugate of the state vector.

        Returns
        -------
        np.ndarray
            Hermitian conjugate of the state vector.
        """
        return self._stateVec.H

    def inv(self) -> cp.ndarray:
        """Returns the inverse of the state vector.

        Returns
        -------
        np.ndarray
            Inverse of the state vector.
        """
        return inv(self._stateVec)

    def resetState(self) -> None:
        """Resets the components of the State."""
        self._stateVec = cp.zeros((self._n, 1), dtype=complex)

    def setDim(self, newDim: int, newNodeList: list = None) -> None:
        """Sets the current state dimension to a user defined one.

        Parameters
        ----------
        newDim : int
            New state dimension.
        newNodeList : list, optional
            List containing the new nodes, by default None.
        """
        self._n = newDim
        self._stateVec = cp.zeros((self._n, 1), dtype=complex)
        if newNodeList is not None:
            self._nodeList = newNodeList

    def getDim(self) -> int:
        """Gets the current state dimension.

        Returns
        -------
        int
            State dimension.
        """
        return self._n

    def setNodeList(self, newNodeList: list) -> None:
        """Sets current node list to a user inputted one.

        Parameters
        ----------
        newNodeList : list
            List containing the new nodes.
        """
        self._nodeList = newNodeList

    def getNodeList(self) -> list:
        """Gets the current list of nodes.

        Returns
        -------
        list
            Current list of nodes.
        """
        return self._nodeList

    def setStateVec(self, newVec: cp.ndarray) -> None:
        """Sets the column vector associated with the state to a user defined one.

        Parameters
        ----------
        newVec : np.ndarray
            New column vector for the state.
        """
        self._stateVec = newVec

    def getStateVec(self) -> cp.ndarray:
        """Gets the column vector associated with the state.

        Returns
        -------
        np.ndarray
            Vector of the State.
        """
        return self._stateVec

    def setState(self, newState: State) -> None:
        """Sets all the parameters of the current state to user defined ones.

        Parameters
        ----------
        newState : State
            New state.
        """
        self._n = newState.getDim()
        self._nodeList = newState.getNodeList()
        self._stateVec = newState.getStateVec()


    def __mul__(self, other: cp.ndarray) -> cp.ndarray:
        """Left-side multiplication for the State class.

        Parameters
        ----------
        other : np.ndarray
            Another Numpy ndarray to multiply the state by.

        Returns
        -------
        np.ndarray
            Array of the multiplication
        """
        return self._stateVec * other

    def __rmul__(self, other: cp.ndarray) -> cp.ndarray:
        """Left-side multiplication for the State class.

        Parameters
        ----------
        other : np.ndarray
            Another Numpy ndarray to multiply the state by.

        Returns
        -------
        np.ndarray
            Array of the multiplication.
        """
        return other * self._stateVec

    def __matmul__(self, other: cp.ndarray) -> cp.ndarray:
        """Matrix multiplication for the State class.

        Parameters
        ----------
        other : np.ndarray
            Another Numpy ndarray to multiply the state by.

        Returns
        -------
        np.ndarray
            Array of the multiplication.
        """
        return self._stateVec @ other

    def __str__(self) -> str:
        """String representation of the State class.

        Returns
        -------
        str
            State string.
        """
        return f"{self._stateVec}"

    def __repr__(self) -> str:
        """String representation of the State class.

        Returns
        -------
        str
            State string.
        """
        return f"N: {self._n}\n" \
               f"Node list: {self._nodeList}\n" \
               f"Custom Node list: {self._customStateList}\n" \
               f"State:\n\t{self._stateVec}"

### Numpy State

In [7]:
from qwak.State import State

n = 4
initNodes = [0,1,2,3]

initState = State(n,initNodes)
initState.buildState()
print(initState.getStateVec())

[[0.5+0.j]
 [0.5+0.j]
 [0.5+0.j]
 [0.5+0.j]]


### Cupy State

In [8]:
n = 4
initNodes = [0,1,2,3]

initState = CupyState(n=n,nodeList=initNodes)
initState.buildState()
print(initState.getStateVec())

NodeList :[0 1 2 3]
Type of state:<class 'cupy.ndarray'>
State:0
Type of state:<class 'int'>
Type of stateVec:<class 'cupy.ndarray'>
Type of state:<class 'cupy.ndarray'>
State:1
Type of state:<class 'int'>
Type of stateVec:<class 'cupy.ndarray'>
Type of state:<class 'cupy.ndarray'>
State:2
Type of state:<class 'int'>
Type of stateVec:<class 'cupy.ndarray'>
Type of state:<class 'cupy.ndarray'>
State:3
Type of state:<class 'int'>
Type of stateVec:<class 'cupy.ndarray'>
[[0.5+0.j]
 [0.5+0.j]
 [0.5+0.j]
 [0.5+0.j]]


In [9]:
test = cp.array([1,2,3])
for el in test:
    print(el)
    print(type(el))

1
<class 'cupy.ndarray'>
2
<class 'cupy.ndarray'>
3
<class 'cupy.ndarray'>


In [10]:
class QWAKState:
    def __init__(self, n: int, nodeList: list = None,
                 customStateList: list = None) -> None:
        """Object is initialized with a mandatory user inputted dimension, an optional
        stateList parameter which will be used to create the amplitudes for each node in the state
        and an internal stateVec which will be a Numpy ndarray representing the column vector.

        Parameters
        ----------
        n : int
            Desired dimension of the state.
        nodeList : list, optional
            List containing what nodes will have uniform superposition in the state, by default None.
        customStateList : list, optional
            Custom amplitudes for the state, by default None.
        """
        self._n = n
        if nodeList is None:
            self._nodeList = []
        else:
            self._nodeList = nodeList
        if customStateList is None:
            self._customStateList = []
        else:
            self._customStateList = customStateList
        self._stateVec = np.zeros((self._n, 1), dtype=complex)

    def buildState(
            self,
            nodeList: list = None,
            customStateList: list = None) -> None:
        """Builds state vector from state list, by creating a balanced superposition of all
        nodes in the nodeList.
        This will be changed in the future to make nodeList make more sense.

        Parameters
        ----------
        nodeList : list, optional
            List containing what nodes will have uniform superposition in the state, by default None.
        customStateList : list, optional
            Custom amplitudes for the state, by default None.
        """
        if nodeList is not None:
            self.resetState()
            self._nodeList = nodeList
        if customStateList is not None:
            self.resetState()
            self._customStateList = customStateList
        if self._customStateList:
            self._checkUnitaryStateList(self._customStateList)
            for customState in self._customStateList:
                self._checkStateOutOfBounds(customState[0])
                self._stateVec[customState[0]] = customState[1]
        else:
            nodeAmp = np.sqrt(len(self._nodeList))
            for state in self._nodeList:
                self._checkStateOutOfBounds(state)
                print(f"Type of state:{type(state)}")
                print(f"Type of stateVec:{type(self._stateVec)}")
                self._stateVec[state] = 1 / nodeAmp

    def _checkStateOutOfBounds(self, node: int) -> None:
        """Checks if the state is out of bounds for the system.

        Parameters
        ----------
        node : int
            Node to check.

        Raises
        ------
        StateOutOfBounds
            Out of bounds exception.
        """
        if node >= self._n:
            raise StateOutOfBounds(
                f"State {node} is out of bounds for system of size {self._n} ([0-{self._n - 1}])."
            )

    def _checkUnitaryStateList(self, customStateList) -> None:
        """Checks if the sum of the square of the amplitudes is 1.

        Parameters
        ----------
        customStateList : list
            Custom state list.

        Raises
        ------
        NonUnitaryState
            Non unitary state exception.
        """
        unitaryState = 0
        for state in customStateList:
            unitaryState += np.abs(state[1]) ** 2
        unitaryState = round(unitaryState, 5)
        if unitaryState != float(1):
            raise NonUnitaryState(
                f"The sum of the square of the amplitudes is -- {unitaryState} -- instead of 1."
            )

    def herm(self) -> np.ndarray:
        """Returns the Hermitian conjugate of the state vector.

        Returns
        -------
        np.ndarray
            Hermitian conjugate of the state vector.
        """
        return self._stateVec.H

    def inv(self) -> np.ndarray:
        """Returns the inverse of the state vector.

        Returns
        -------
        np.ndarray
            Inverse of the state vector.
        """
        return inv(self._stateVec)

    def resetState(self) -> None:
        """Resets the components of the State."""
        self._stateVec = np.zeros((self._n, 1), dtype=complex)

    def setDim(self, newDim: int, newNodeList: list = None) -> None:
        """Sets the current state dimension to a user defined one.

        Parameters
        ----------
        newDim : int
            New state dimension.
        newNodeList : list, optional
            List containing the new nodes, by default None.
        """
        self._n = newDim
        self._stateVec = np.zeros((self._n, 1), dtype=complex)
        if newNodeList is not None:
            self._nodeList = newNodeList

    def getDim(self) -> int:
        """Gets the current state dimension.

        Returns
        -------
        int
            State dimension.
        """
        return self._n

    def setNodeList(self, newNodeList: list) -> None:
        """Sets current node list to a user inputted one.

        Parameters
        ----------
        newNodeList : list
            List containing the new nodes.
        """
        self._nodeList = newNodeList

    def getNodeList(self) -> list:
        """Gets the current list of nodes.

        Returns
        -------
        list
            Current list of nodes.
        """
        return self._nodeList

    def setStateVec(self, newVec: np.ndarray) -> None:
        """Sets the column vector associated with the state to a user defined one.

        Parameters
        ----------
        newVec : np.ndarray
            New column vector for the state.
        """
        self._stateVec = newVec

    def getStateVec(self) -> np.ndarray:
        """Gets the column vector associated with the state.

        Returns
        -------
        np.ndarray
            Vector of the State.
        """
        return self._stateVec

    def setState(self, newState: State) -> None:
        """Sets all the parameters of the current state to user defined ones.

        Parameters
        ----------
        newState : State
            New state.
        """
        self._n = newState.getDim()
        self._nodeList = newState.getNodeList()
        self._stateVec = newState.getStateVec()

    def to_json(self) -> str:
        """In contrast, the to_json method is not marked with the @classmethod decorator because
        it is a method that is called on an instance of the Operator class.

        This means that it can access the attributes of the instance on which it is called, and it
        uses these attributes to generate the JSON string representation of the Operator instance.

        Since it requires access to the attributes of a specific Operator instance, it cannot be
        called on the Operator class itself.

        Returns
        -------
        str
            JSON string representation of the Operator instance.
        """
        state_dict = {
            "n": self._n,
            "node_list": self._nodeList,
            "custom_state_list": self._customStateList,
            "state_vec": complex_matrix_to_json(
                self._stateVec.tolist()),
        }
        return json.dumps(state_dict)

    @classmethod
    def from_json(cls, json_var: Union[str, dict]):
        """The from_json method is marked with the @classmethod decorator because it is a method that is called on the class itself,
        rather than on an instance of the class.

        This is necessary because it is used to create a new instance of the Operator class from a JSON string,
        and it does not require an instance of the Operator class to do so.

        Parameters
        ----------
        json_var : Union([str, dict])
            JSON string or dictionary representation of the Operator instance.

        Returns
        -------
        Operator
            Operator instance from JSON string or dictionary representation.
        """
        if isinstance(json_var, str):
            state_dict = json.loads(json_var)
        elif isinstance(json_var, dict):
            state_dict = json_var
        n = state_dict["n"]
        node_list = state_dict["node_list"]
        custom_state_list = state_dict["custom_state_list"]
        state_vec = np.array(
            json_matrix_to_complex(
                state_dict["state_vec"]),
            dtype=complex)
        state = cls(n, node_list, custom_state_list)
        state.setStateVec(state_vec)
        return state

    def __mul__(self, other: np.ndarray) -> np.ndarray:
        """Left-side multiplication for the State class.

        Parameters
        ----------
        other : np.ndarray
            Another Numpy ndarray to multiply the state by.

        Returns
        -------
        np.ndarray
            Array of the multiplication
        """
        return self._stateVec * other

    def __rmul__(self, other: np.ndarray) -> np.ndarray:
        """Left-side multiplication for the State class.

        Parameters
        ----------
        other : np.ndarray
            Another Numpy ndarray to multiply the state by.

        Returns
        -------
        np.ndarray
            Array of the multiplication.
        """
        return other * self._stateVec

    def __matmul__(self, other: np.ndarray) -> np.ndarray:
        """Matrix multiplication for the State class.

        Parameters
        ----------
        other : np.ndarray
            Another Numpy ndarray to multiply the state by.

        Returns
        -------
        np.ndarray
            Array of the multiplication.
        """
        return self._stateVec @ other

    def __str__(self) -> str:
        """String representation of the State class.

        Returns
        -------
        str
            State string.
        """
        return f"{self._stateVec}"

    def __repr__(self) -> str:
        """String representation of the State class.

        Returns
        -------
        str
            State string.
        """
        return f"N: {self._n}\n" \
               f"Node list: {self._nodeList}\n" \
               f"Custom Node list: {self._customStateList}\n" \
               f"State:\n\t{self._stateVec}"


In [11]:
initState = QWAKState(n,initNodes)
initState.buildState()
print(initState.getStateVec())

Type of state:<class 'int'>
Type of stateVec:<class 'numpy.ndarray'>
Type of state:<class 'int'>
Type of stateVec:<class 'numpy.ndarray'>
Type of state:<class 'int'>
Type of stateVec:<class 'numpy.ndarray'>
Type of state:<class 'int'>
Type of stateVec:<class 'numpy.ndarray'>
[[0.5+0.j]
 [0.5+0.j]
 [0.5+0.j]
 [0.5+0.j]]


# Operator

In [12]:
from __future__ import annotations
from typing import Union

import networkx as nx
import sympy as sp
import cupyx.scipy.linalg as cpx_scipy
from sympy.abc import pi
import cupy as cp
from utils.jsonTools import json_matrix_to_complex, complex_matrix_to_json
import json

from qwak.Errors import MissingNodeInput
from utils.PerfectStateTransfer import isStrCospec, checkRoots, swapNodes, getEigenVal

class CupyOperator:
    def __init__(
            self,
            graph: nx.Graph,
            gamma: float = 1,
            time: float = 0,
            laplacian: bool = False,
            markedElements: list = [],
    ) -> None:
        """
        Class for the quantum walk operator.

        This object is initialized with a user inputted graph, which is then used to
        generate the dimension of the operator and the adjacency matrix, which is
        the central structure required to perform walks on regular graphs. Note that this
        version of the software only supports regular undirected graphs, which will hopefully
        be generalized in the future.

        The eigenvalues and eigenvectors of the adjacency matrix are also calculated during
        initialization, which are then used to calculate the diagonal operator through spectral
        decomposition. This was the chosen method since it is computationally cheaper than calculating
        the matrix exponent directly.

        Parameters
        ----------
        graph : nx.Graph
            Graph where the walk will be performed.
        gamma : float
            Needs Completion.
        time: float, optional
            Time for which to calculate the operator, by default None.
        laplacian : bool, optional
            Allows the user to choose whether to use the Laplacian or simple adjacency matrix, by default False.
        markedElements : list, optional
            List with marked elements for search, by default None.
        """
        self._time = time
        self._gamma = gamma
        self._laplacian = laplacian
        self._markedElements = markedElements
        self._graph = graph
        self._n = len(graph)
        self._operator = cp.zeros((self._n, self._n), dtype=complex)

        self._hamiltonian = self._buildHamiltonian(self._graph,self._laplacian)
        if self._markedElements:
            self._hamiltonian = self._buildSearchHamiltonian(self._hamiltonian, self._markedElements)

        self._isHermitian = self._hermitianTest(self._hamiltonian)
        self._eigenvalues, self._eigenvectors = self._buildEigenValues(self._hamiltonian)

    def buildDiagonalOperator(self, time: float = 0) -> None:
        """Builds operator matrix from optional time and transition rate parameters, defined by user.

        The first step is to calculate the diagonal matrix that takes in time, transition rate and
        eigenvalues and convert it to a list of the diagonal entries.

        The entries are then multiplied
        by the eigenvectors, and the last step is to perform matrix multiplication with the complex
        conjugate of the eigenvectors.

        Parameters
        ----------
        time : float, optional
            Time for which to calculate the operator, by default 0.
        gamma : float, optional
            Needs completion.
        round : int, optional
        """
        self._time = time
        diag = cp.diag(
            cp.exp(-1j * self._eigenvalues * self._time)).diagonal()
        self._operator = cp.multiply(self._eigenvectors, diag)
        if self._isHermitian:
            self._operator = cp.matmul(
                    self._operator, self._eigenvectors.conjugate().transpose())
        else:
            self._operator = cp.matmul(
                    self._operator, inv(
                    self._eigenvectors))

    def buildExpmOperator(self, time: float = 0) -> None:
        """Builds operator matrix from optional time and transition rate parameters, defined by user.

        Uses the scipy function expm to calculate the matrix exponential of the adjacency matrix.

        Parameters
        ----------
        time : float, optional
            Time for which to calculate the operator, by default 0.
        """
        self._time = time
        self._operator = cpx_scipy.expm(-1j * self._hamiltonian * self._time)

    def _buildHamiltonian(
            self,
            graph,
            laplacian: bool,
                    ) -> cp.ndarray:
        """Builds the hamiltonian of the graph, which is either the Laplacian or the simple matrix.

        Parameters
        ----------
        laplacian : bool
            Allows the user to choose whether to use the Laplacian or simple adjacency matrix.
        markedElements : list
            List of elements for the search.
        """
        self._adjacency = cp.array(nx.to_numpy_array(
            graph, dtype=complex))
        if laplacian:
            self._adjacency = self._adjacency - self._degreeDiagonalMatrix(graph)
        return -self._adjacency * self._gamma

    def _buildSearchHamiltonian(self,hamiltonian,markedElements):
        for marked in markedElements:
            hamiltonian[marked[0], marked[0]] += marked[1]
        return hamiltonian

    def _buildEigenValues(self, hamiltonian) -> None:
        """Builds the eigenvalues and eigenvectors of the adjacency matrix.

        Parameters
        ----------
        isHermitian : bool
            Checks if the adjacency matrix is Hermitian.
        """

        if self._isHermitian:
            eigenvalues, eigenvectors = cp.linalg.eigh(
                hamiltonian
            )
        else:
            eigenvalues, eigenvectors  = cp.linalg.eig(
                hamiltonian )
        return eigenvalues, eigenvectors

    def _hermitianTest(self, hamiltonian) -> bool:
        """Checks if the adjacency matrix is Hermitian.

        Parameters
        ----------
        hamiltonian : cp.ndarray
            Adjacency matrix.

        Returns
        -------
        bool
            True if Hermitian, False otherwise.
        """
        return cp.allclose(hamiltonian, hamiltonian.conj().T)

    def getEigenValues(self) -> list:
        """Returns the eigenvalues of the adjacency matrix.

        Returns
        -------
        list
            List of eigenvalues.
        """
        return self._eigenvalues

    def _setEigenValues(self, eigenValues: list) -> None:
        """Sets the eigenvalues of the adjacency matrix.

        Parameters
        ----------
        eigenValues : list
            List of eigenvalues.
        """
        self._eigenvalues = eigenValues

    def getEigenVectors(self) -> list:
        """Returns the eigenvectors of the adjacency matrix.

        Returns
        -------
        list
            List of eigenvectors.
        """
        return self._eigenvectors

    def _setEigenVectors(self, eigenVectors: list) -> None:
        """Sets the eigenvectors of the adjacency matrix.

        Parameters
        ----------
        eigenVectors : list
            _description_
        """
        self._eigenvectors = eigenVectors

    def getHamiltonian(self):
        """Returns the hamiltonian of the graph, which is either the Laplacian or the simple matrix.

        Returns
        -------
        cp.ndarray
            Hamiltonian of the graph.
        """
        return self._hamiltonian

    def setHamiltonian(self, hamiltonian):
        """Sets the hamiltonian for the walk.

        Parameters
        ----------
        hamiltonian : cp.ndarray
            Hamiltonian of the graph.
        """
        self._hamiltonian = hamiltonian
        self._eigenvalues, self._eigenvectors = self._buildEigenValues(self._hamiltonian)

    def resetOperator(self) -> None:
        """Resets Operator object."""
        self._operator = cp.zeros((self._n, self._n), dtype=complex)

    def setDim(self, newDim: int, graph: nx.Graph) -> None:
        """Sets the current Operator objects dimension to a user defined one.

        Parameters
        ----------
        newDim : int
            New dimension for the Operator object.
        graph : nx.Graph
            New graph for the Operator object.
        """
        self._n = newDim
        self._operator = cp.zeros((self._n, self._n), dtype=complex)
        self._graph = graph
        self._hamiltonian = (
            cp.array(nx.adjacency_matrix(self._graph).todense(), dtype=complex)
        )
        self._adjacency = self._hamiltonian
        self.setAdjacencyMatrix(self._hamiltonian)

    def getDim(self) -> int:
        """Gets the current graph dimension.

        Returns
        -------
        int
            Dimension of Operator object.
        """
        return self._n

    def setTime(self, newTime: float) -> None:
        """Sets the current operator time to a user defined one.

        Parameters
        ----------
        newTime : float
            New operator time.
        """
        self._time = newTime

    def getTime(self) -> float:
        """Gets the current operator time.

        Returns
        -------
        float
            Current time of Operator object.
        """
        return self._time

    def setAdjacencyMatrix(self, adjacencyMatrix: cp.ndarray) -> None:
        """Sets the adjacency matrix of the operator to a user defined one.
        Might make more sense to not give the user control over this parameter, and make
        them instead change the graph entirely.

        Parameters
        ----------
        adjacencyMatrix : cp.ndarray
            New adjacency matrix for the Operator object.
        """
        self._hamiltonian = adjacencyMatrix.astype(complex)
        self._adjacency = self._hamiltonian
        self._n = len(self._hamiltonian)
        self.resetOperator()
        self._eigenvalues, self._eigenvectors = self._buildEigenValues(self._hamiltonian)
        
        
    def _setAdjacencyMatrixOnly(
            self, adjacencyMatrix: cp.ndarray) -> None:
        """Sets the adjacency matrix of the operator to a user defined one.
        Might make more sense to not give the user control over this parameter, and make
        them instead change the graph entirely.

        Parameters
        ----------
        adjacencyMatrix : cp.ndarray
            New adjacency matrix.
        """
        self._hamiltonian = adjacencyMatrix.astype(complex)
        self._n = len(self._hamiltonian)
        self.resetOperator()
        
    def getAdjacencyMatrix(self) -> cp.ndarray:
        """Gets the current adjacency matrix of the Operator.

        Returns
        -------
        cp.ndarray
            Adjacency matrix of the Operator.
        """
        return self._adjacency
    
    def _setOperatorVec(self, newOperator: cp.ndarray) -> None:
        """Sets all the parameters of the current operator to user defined ones.

        Parameters
        ----------
        newOperator : Operator
            New user inputted Operator.
        """
        self._operator = newOperator
        
    def setOperator(self, newOperator: Operator) -> None:
        """Sets all the parameters of the current operator to user defined ones.

        Parameters
        ----------
        newOperator : Operator
            New user inputted Operator.
        """
        self._n = newOperator.getDim()
        self._time = newOperator.getTime()
        self._operator = newOperator.getOperator()

    def getOperator(self) -> cp.ndarray:
        """Gets the cupy ndarray associated with the current operator.

        Returns
        -------
        cp.ndarray
            Current Operator object.
        """
        return self._operator
    
    def getMarkedElements(self) -> list:
        """Returns the marked elements of the operator.

        Returns
        -------
        list
            List of marked elements.
        """
        return self._markedElements
    
    def setMarkedElements(self, markedElements: list) -> None:
        """Sets the marked elements of the operator.

        Parameters
        ----------
        markedElements : list
            List of marked elements.
        """
        self._markedElements = markedElements
        
    @staticmethod
    def _degreeDiagonalMatrix(G):
        degrees = cp.array(list(dict(G.degree()).values()))
        return cp.diag(degrees)
 
    def __mul__(self, other: cp.ndarray) -> cp.ndarray:
        return self._operator * other

    def __rmul__(self, other: cp.ndarray) -> cp.ndarray:
        return other * self._operator

    def __str__(self) -> str:
        return f"{self._operator}"

    def __repr__(self) -> str:
        """Representation of the ProbabilityDistribution object.

        Returns
        -------
        str
            String of the ProbabilityDistribution object.
        """
        return f"N: {self._n}\n" \
               f"Time: {self._time}\n" \
               f"Graph: {nx.to_dict_of_dicts(self._graph)}\n" \
               f"Operator:\n\t{self._operator}"
    

### Numpy Operator

In [13]:
from qwak.Operator import Operator

n = 4
graph = nx.cycle_graph(n)

operator = Operator(graph)
operator.buildDiagonalOperator(0.5)
print(operator.getOperator().round(3))

[[ 0.77+0.j     0.  +0.421j -0.23+0.j     0.  +0.421j]
 [ 0.  +0.421j  0.77+0.j     0.  +0.421j -0.23+0.j   ]
 [-0.23+0.j     0.  +0.421j  0.77+0.j     0.  +0.421j]
 [ 0.  +0.421j -0.23+0.j     0.  +0.421j  0.77+0.j   ]]


### Cupy Operator

In [14]:
n = 4
graph = nx.cycle_graph(n)

operator = CupyOperator(graph)
operator.buildDiagonalOperator(0.5)
print(operator.getOperator().round(3))

[[ 0.77+0.j     0.  +0.421j -0.23+0.j     0.  +0.421j]
 [ 0.  +0.421j  0.77-0.j     0.  +0.421j -0.23-0.j   ]
 [-0.23+0.j     0.  +0.421j  0.77+0.j     0.  +0.421j]
 [ 0.  +0.421j -0.23-0.j     0.  +0.421j  0.77+0.j   ]]
